# LOAD MODEL

In [1]:
MODEL_PATH = 'knn.sav'

In [2]:
import pickle
model = pickle.load(open(MODEL_PATH, 'rb'))

# RUN LIVE

In [4]:
import cv2
import mediapipe as mp
import numpy as np

objc[19985]: Class CaptureDelegate is implemented in both /Users/prakhar/opt/miniconda3/envs/ML/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x179d52538) and /Users/prakhar/opt/miniconda3/envs/ML/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x17b6fc860). One of the two will be used. Which one is undefined.
objc[19985]: Class CVWindow is implemented in both /Users/prakhar/opt/miniconda3/envs/ML/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x179d52588) and /Users/prakhar/opt/miniconda3/envs/ML/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x177e6ca68). One of the two will be used. Which one is undefined.
objc[19985]: Class CVView is implemented in both /Users/prakhar/opt/miniconda3/envs/ML/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x179d525b0) and /Users/prakhar/opt/miniconda3/envs/ML/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x177e6ca90). One of the two will be used. Which one i

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [13]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success: continue

    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    pose_landmarks = results.pose_landmarks

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    image = cv2.flip(image, 1)

    if pose_landmarks is not None:
      pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
      frame_height, frame_width = image.shape[:2]
      pose_landmarks *= np.array([frame_width, frame_height, frame_width])
      pose_landmarks = np.around(pose_landmarks, 5).flatten()
      pose_landmarks = pose_landmarks.reshape(1, -1)

      prediction = model.predict(pose_landmarks)

      cv2.putText(img=image, text=str(prediction), org=(150, 250), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=3, color=(0, 0, 0),thickness=3)

  
  
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Pose', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

: 